In [ ]:
import pandas as pd
import numpy as np

In [113]:
submit = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/jiwhan/2-2_검증데이터셋.csv', encoding='cp949')
hospital = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/jiwhan/hospital.csv', encoding='cp949')
new_merge_data = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/jiwhan/new_merge_data.csv', encoding='cp949')
forecast = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/jiwhan/예보지수.csv', encoding='cp949')

def add_lag_feature(dat, lag = None):

    cols = [
            'avg_tca_max', 'sum_rn_max',
            'sum_ss_hr_max', 'ssrate_max', 'avg_rhm_max', 'min_rhm_max',
            'max_ws_max','avg_ws_max','max_ta_max', 'min_ta_max', 'PM10_max']
    '''
    cols = ['avg_tca_mean','sum_ss_hr_mean', 'ssrate_mean','avg_rhm_mean', 'min_rhm_mean','sum_rn_mean', 
            'avg_ws_mean', 'max_ws_mean', 'avg_ta_mean', 'max_ta_mean', 'min_ta_mean', 'PM10_mean']
            
    cols = ['avg_tca_mean', 'avg_tca_max','sum_ss_hr_mean','sum_ss_hr_max', 'ssrate_mean', 
            'ssrate_max', 'avg_rhm_mean','avg_rhm_max', 'min_rhm_mean', 'min_rhm_max', 
            'sum_rn_mean', 'sum_rn_max', 'avg_ws_mean', 'avg_ws_max', 'max_ws_mean', 
            'max_ws_max','avg_ta_mean', 'avg_ta_max','max_ta_mean', 'max_ta_max', 
            'min_ta_mean', 'min_ta_max','PM10_mean','PM10_max']
    '''
    
    group_df = dat.groupby(['area'])
    lag_mean = group_df[cols].rolling(window = lag, min_periods=0, closed='left').mean().reset_index(0,drop=True)
   # lag_max = group_df[cols].rolling(window = lag, min_periods=0).max().reset_index(0,drop=True)
    for col in cols:
        dat[f'{col}/mean_lag{lag}'] = lag_mean[col]
   #     dat[f'{col}/max_lag{lag}'] = lag_max[col]
    return dat

# new_merge_data = add_lag_feature(new_merge_data.copy(), 2)
new_merge_data = add_lag_feature(new_merge_data.copy(), 3)
# new_merge_data = add_lag_feature(new_merge_data.copy(), 4)
# new_merge_data = add_lag_feature(new_merge_data.copy(), 5)
# new_merge_data = add_lag_feature(new_merge_data.copy(), 6)
# new_merge_data = add_lag_feature(new_merge_data.copy(), 7)


final = pd.merge(hospital, new_merge_data, how='left')
final = pd.merge(final, forecast, how='left')
final = final.sort_values(['yyyymmdd', 'sex', 'area']).reset_index(drop=True)

# 새로운 변수 생성 
dates = pd.to_datetime(final['yyyymmdd'], format='%Y%m%d').dt      
final['weekday']= dates.weekday                                     # 요일 
final['risk_ratio'] = final['num_risk_age'] / final['tot_person']   # 위험인구 비율 
final['day_diff'] = final['max_ta_mean'] - final['min_ta_mean']     # 일교차 
final['temp_humi'] = final['avg_rhm_mean'] * final['avg_ta_mean']   # 습도 * 기온

final['temp_threshold'] = np.where((final['min_ta_mean'] < -1.5), 1, 0)  # 뇌졸중 영향 온도 threshhold

conditionlist = [
    (final['month'].isin([3,4,5])),
    (final['month'].isin([6,7,8])),
    (final['month'].isin([9,10,11])),
    (final['month'].isin([12,1,2]))]
choicelist = [1,2,3,4]
final['season'] = np.select(conditionlist, choicelist, default='Not Specified')  # 계절
final['season'] = final['season'].astype(int)
final['diff_risk'] = np.where(final['day_diff'] >= 10, 1, 0)                     # 일교차 10도 이상 



def make_month_category(df,target_col,new_col_name,sign):
  mean_data = list(df.groupby(['month'])[target_col].mean())

  # 이상의 값을 구할때
  if sign == '>=':
    for year in range(2012,2016+1):
      for month in range(1,12+1):
        if (df.loc[(df['year']==year) & (df['month']==month),target_col].mean()) >= mean_data[month-1]:
          df.loc[(df['year']==year) & (df['month']==month),new_col_name] = 1
        else:
          df.loc[(df['year']==year) & (df['month']==month),new_col_name] = 0
  # 이하의 값을 구할때
  else:
    for year in range(2012,2016+1):
      for month in range(1,12+1):
        if (df.loc[(df['year']==year) & (df['month']==month),target_col].mean()) <= mean_data[month-1]:
          df.loc[(df['year']==year) & (df['month']==month),new_col_name] = 1
        else:
          df.loc[(df['year']==year) & (df['month']==month),new_col_name] = 0   

  return df

# 새로운 변수 추가
# 2012 ~ 2015년 동안의 월 평균 환자 수 의 평균값보다 많은 달은 risk_patient_month = 1 ,아니면 risk_patient_month = 0
final = make_month_category(final.copy(),'frequency','risk_patient_month','>=')

# 새로운 변수 추가
# 2012 ~ 2015년 동안 월 평균 기온보다 연도별,월별 평균 기온이 낮다면 risk_ta_month = 1, 아니면 0
final = make_month_category(final.copy(),'avg_ta_mean','risk_ta_month','<=')

# 새로운 변수 추가
# 2012 ~ 2015년 동안 월 평균 습도보다 연도별,월별 평균 습도가 낮다면 risk_ta_month = 1, 아니면 0
final = make_month_category(final.copy(),'avg_rhm_mean','risk_rhm_month','<=')




final = final.astype({'risk_patient_month':'int64','risk_ta_month':'int64','risk_rhm_month':'int64'})


final['risk_month'] = (final['risk_ta_month'] | final['risk_rhm_month'])

# # df1만 가지고 있는 열로 구성된 dataframe을 반환하는 함수
# def get_difference_col(df1,df2):
#   col_original = list(df1.columns)
#   col_remove = list(df2.columns)
#   col_target = [i for i in col_original if i not in col_remove]
#   res_df = df1[col_target]
#   del df1,df2
#   return res_df

# final_ver1 = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_res.csv', encoding='cp949')
# final_ver3 = pd.concat([final,get_difference_col(final_ver1,final)],axis=1)

# final_ver3.to_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_ver3.csv',encoding='cp949')
final.to_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_ver4.csv',encoding='cp949')

In [89]:
final['risk_month'].value_counts()

1    21896
2    21658
0    18564
Name: risk_month, dtype: int64

# 모델링

In [ ]:
! pip install catboost
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 30.7 MB/s 
     |████████████████████████████████| 209 kB 42.9 MB/s 
     |████████████████████████████████| 81 kB 11.6 MB/s 
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 112 kB 54.9 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
     |████████████████████████████████| 147 kB 51.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=0849e31be8802285d413ab1a43f21fae4abd1e2f3645db58f8003a4d88dca1e3
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import RobustScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold, KFold
import random
random.seed(42)
np.random.seed(42)

# 모델 평가 지표 
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

In [130]:
final = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_ver4.csv',encoding='cp949',index_col=0)

cat_features = ['area']
for i in enumerate(cat_features) :
    ca = i[1]
    final[ca] = final[ca].astype('category')

# 레이블 인코딩된 categorical data의 타입을 int형으로 변환
final = final.astype({'sex':'int64', 'year':'int64', 'month':'int64', 'day':'int64', 'day_differ':'int64','month_differ':'int64', 
                      'weekday':'int64', 'temp_threshold':'int64', 'season':'int64', 'diff_risk':'int64'})

# column drop 하는 부분
final = final.drop([
'A03_t1_max',
 'A03_t2_max',
 'A03_t3_max',
 'A04_t1_max',
 'A04_t2_max',
 'A04_t3_max',
 'A05_t1_max',
 'A05_t2_max',
 'A05_t3_max',
 'A06_t1_max',
 'A06_t2_max',
 'A06_t3_max',
 'A07_t1_max',
 'A07_t2_max',
 'A07_t3_max',
 'A07_t3_mean',
 'CO_max',
 'NO2_max',
 'O3_max',
 'PM10_max',
 'PM10_max/mean_lag3',
 'SO2_max',
 'avg_rhm_max',
 'avg_rhm_max/mean_lag3',
 'avg_ta_max',
 'avg_tca_max',
 'avg_tca_max/mean_lag3',
 'avg_tca_mean',
 'avg_ws_max',
 'avg_ws_max/mean_lag3',
 'max_ta_max',
 'max_ta_max/mean_lag3',
 'max_ta_mean',
 'max_ws_max',
 'max_ws_max/mean_lag3',
 'max_ws_mean',
 'min_rhm_max',
 'min_rhm_max/mean_lag3',
 'min_ta_max',
 'min_ta_max/mean_lag3',
 'num_risk_age',
 'risk_rhm_month',
 'risk_ta_month',
 'ssrate_max',
 'ssrate_max/mean_lag3',
 'sum_rn_max',
 'sum_rn_max/mean_lag3',
 'sum_rn_mean',
 'sum_ss_hr_max',
 'sum_ss_hr_max/mean_lag3',
 
 
 'risk_ta_month',
 'risk_rhm_month',
 'num_risk_age',
'A07_t3_max','A07_t3_mean'], axis=1)

# train, test split
train = final[final['year'] != 2016].drop('yyyymmdd', axis=1)
test = final[final['year'] == 2016].drop(['frequency', 'yyyymmdd'], axis=1)

X = train.drop('frequency', axis=1)
y = train['frequency']

# LGBM 단일 모델

In [131]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42, stratify=X['month'])

# 모델링 
LGBM = LGBMRegressor(objective='mse', verbose=1, metric='rmse', random_state=42, n_jobs=-1,
                     learning_rate=0.05, n_estimators = 3000, max_depth = 100)
LGBM.fit(X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds = 300, verbose=False)

score = RMSE(y_val, LGBM.predict(X_val))
print(score)

1.1903706018186229


# CATBoost 단일 모델

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42, stratify=X['month'])

# 모델링 
CAT = CatBoostRegressor(verbose=2, random_state=42, one_hot_max_size = 17)
CAT.fit(X_train, y_train,
        cat_features=['area', 'sex', 'year', 'month', 'day', 'day_differ', 
                      'month_differ', 'weekday', 'temp_threshold', 'season', 'diff_risk','risk_patient_month','risk_month'],
        eval_set=(X_val, y_val),
        early_stopping_rounds = 300, verbose=False)
score = RMSE(y_val, CAT.predict(X_val))
print(score)

# LGBM k-fold

In [111]:
LGBM = LGBMRegressor(objective='mse', verbose=1, metric='rmse', random_state=42, n_jobs=-1,
                     learning_rate=0.05, n_estimators = 3000, max_depth = 100)
# K-Fold 
def kfold(model, tr, num):
    cv_accuracy = []
    cv = KFold(n_splits = num)
    
    n_iter = 0

    for t,v in cv.split(tr, tr['frequency']): 
        train_cv = tr.iloc[t]
        val_cv = tr.iloc[v]

        X = train_cv.drop('frequency', axis=1)
        y = train_cv['frequency']

        val_X = val_cv.drop('frequency', axis=1)
        val_y = val_cv['frequency']

        model.fit(X, y, eval_set=(val_X, val_y), early_stopping_rounds=200, verbose=False)
        score = RMSE(val_y, model.predict(val_X))
        print(f'{n_iter+1} rmse ======> {score}')

        cv_accuracy.append(score)
        n_iter += 1
    
    return np.mean(cv_accuracy)
kfold(LGBM, train, 10)

1 rmse ======> 1.2299361043260604
2 rmse ======> 1.1937339047786095
3 rmse ======> 1.1876877415341265
4 rmse ======> 1.2287292977821573
5 rmse ======> 1.1898091962651767
6 rmse ======> 1.2038868308729243
7 rmse ======> 1.1882037709093667
8 rmse ======> 1.2221211033215493
9 rmse ======> 1.2373977592222039
10 rmse ======> 1.2144502751982766


1.209595598421045

# Optuna LGBM 튜닝

In [132]:
# 학습함수 정의 
def objective(trial: Trial):
    param = {
        'objective': 'regression',
        'verbose' : -1,
        'metric' : 'rmse',
        'random_state': 42, 
        'n_jobs': -1,
        'n_estimators' : 3000,
        'num_leaves' : trial.suggest_int('num_leaves', 2, 10, 1),
        'subsample' : trial.suggest_float('subsample', 0.7, 1.0, step=0.05),
        'min_child_samples' : trial.suggest_int('min_child_samples', 2, 150, 2),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 20, 400, 2),
        'reg_alpha' : trial.suggest_float('reg_alpha', 1, 1.5, step = 0.1),
        'reg_lambda' : trial.suggest_float('reg_lambda', 1, 1.5, step = 0.1)
    }
    model = LGBMRegressor(**param, random_seed=42)
    model.fit(X_train, y_train, eval_set=[(X_val,y_val)], categorical_feature = 'auto', verbose=False, early_stopping_rounds=100)
    rmse = RMSE(y_val, model.predict(X_val))

    return rmse

# create study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='LGBMRegressor-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial), n_trials=300)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2022-07-24 11:24:39,522] A new study created in memory with name: LGBMRegressor-Hyperparameter-Tuning
[I 2022-07-24 11:24:40,856] Trial 0 finished with value: 1.1887243847818976 and parameters: {'num_leaves': 5, 'subsample': 1.0, 'min_child_samples': 110, 'min_data_in_leaf': 248, 'reg_alpha': 1.0, 'reg_lambda': 1.0}. Best is trial 0 with value: 1.1887243847818976.
[I 2022-07-24 11:24:42,084] Trial 1 finished with value: 1.206333926333602 and parameters: {'num_leaves': 2, 'subsample': 1.0, 'min_child_samples': 92, 'min_data_in_leaf': 290, 'reg_alpha': 1.0, 'reg_lambda': 1.5}. Best is trial 0 with value: 1.1887243847818976.
[I 2022-07-24 11:24:43,613] Trial 2 finished with value: 1.1880958591688144 and parameters: {'num_leaves': 9, 'subsample': 0.75, 'min_child_samples': 28, 'min_data_in_leaf': 90, 'reg_alpha': 1.1, 'reg_lambda': 1.3}. Best is trial 2 with value: 1.1880958591688144.
[I 2022-07-24 11:24:44,785] Trial 3 finished with value: 1.1887717629353503 and parameters: {'num_leave

Best trial : score 1.1865880960310222, 
params {'num_leaves': 6, 'subsample': 1.0, 'min_child_samples': 130, 'min_data_in_leaf': 150, 'reg_alpha': 1.0, 'reg_lambda': 1.1}


In [ ]:
# 단일 lgbm best : 1.1885111217872646
'''
# column drop 하는 부분
final = final.drop([
'A03_t1_max',
 'A03_t2_max',
 'A03_t3_max',
 'A04_t1_max',
 'A04_t2_max',
 'A04_t3_max',
 'A05_t1_max',
 'A05_t2_max',
 'A05_t3_max',
 'A06_t1_max',
 'A06_t2_max',
 'A06_t3_max',
 'A07_t1_max',
 'A07_t2_max',
 'A07_t3_max',
 'A07_t3_mean',
 'CO_max',
 'NO2_max',
 'O3_max',
 'PM10_max',
 'PM10_max/mean_lag3',
 'SO2_max',
 'avg_rhm_max',
 'avg_rhm_max/mean_lag3',
 'avg_ta_max',
 'avg_tca_max',
 'avg_tca_max/mean_lag3',
 'avg_tca_mean',
#  'avg_ws_max',
#  'avg_ws_max/mean_lag3',
 'max_ta_max',
 'max_ta_max/mean_lag3',
 'max_ta_mean',
#  'max_ws_max',
#  'max_ws_max/mean_lag3',
 'max_ws_mean',
 'min_rhm_max',
 'min_rhm_max/mean_lag3',
 'min_ta_max',
 'min_ta_max/mean_lag3',
 'num_risk_age',
 'risk_rhm_month',
 'risk_ta_month',
 'ssrate_max',
 'ssrate_max/mean_lag3',
 'sum_rn_max',
 'sum_rn_max/mean_lag3',
 'sum_rn_mean',
 'sum_ss_hr_max',
 'sum_ss_hr_max/mean_lag3',

'avg_tca_mean',
# 'avg_rhm_mean',
'sum_rn_mean',
# 'avg_ta_mean',
# 'min_ta_mean',
'PM10_mean',

#  'CO_mean',
#  'NO2_mean',
#  'O3_mean',
#  'PM10_mean',
#  'SO2_mean',

#  'A06_t1_mean',
#  'A06_t2_mean',
#  'A06_t3_mean',
 
#  'A05_t1_mean',
#  'A05_t2_mean',
#  'A05_t3_mean',

#  'A04_t1_mean',
#  'A04_t2_mean',
#  'A04_t3_mean',

#  'A03_t1_mean',
#  'A03_t2_mean',
#  'A03_t3_mean',

], axis=1)
 '''

In [ ]:
# optuna 튜닝 lgbm best : 1.1865880960310222
'''
# column drop 하는 부분
final = final.drop([
'A03_t1_max',
 'A03_t2_max',
 'A03_t3_max',
 'A04_t1_max',
 'A04_t2_max',
 'A04_t3_max',
 'A05_t1_max',
 'A05_t2_max',
 'A05_t3_max',
 'A06_t1_max',
 'A06_t2_max',
 'A06_t3_max',
 'A07_t1_max',
 'A07_t2_max',
 'A07_t3_max',
 'A07_t3_mean',
 'CO_max',
 'NO2_max',
 'O3_max',
 'PM10_max',
 'PM10_max/mean_lag3',
 'SO2_max',
 'avg_rhm_max',
 'avg_rhm_max/mean_lag3',
 'avg_ta_max',
 'avg_tca_max',
 'avg_tca_max/mean_lag3',
 'avg_tca_mean',
 'avg_ws_max',
 'avg_ws_max/mean_lag3',
 'max_ta_max',
 'max_ta_max/mean_lag3',
 'max_ta_mean',
 'max_ws_max',
 'max_ws_max/mean_lag3',
 'max_ws_mean',
 'min_rhm_max',
 'min_rhm_max/mean_lag3',
 'min_ta_max',
 'min_ta_max/mean_lag3',
 'num_risk_age',
 'risk_rhm_month',
 'risk_ta_month',
 'ssrate_max',
 'ssrate_max/mean_lag3',
 'sum_rn_max',
 'sum_rn_max/mean_lag3',
 'sum_rn_mean',
 'sum_ss_hr_max',
 'sum_ss_hr_max/mean_lag3',
 
 
 'risk_ta_month',
 'risk_rhm_month',
 'num_risk_age',
'A07_t3_max','A07_t3_mean'], axis=1)
 '''